In [1]:
pip install requests beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install mysql-connector-python


Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
from bs4 import BeautifulSoup
import mysql.connector
import time


url = 'https://books.toscrape.com/catalogue/page-{}.html'

def get_page_content(page_url):
    response = requests.get(page_url)
    return BeautifulSoup(response.content, 'html.parser') 


def scrape_book_details(book_container):
    
    Title = book_container.find('h3').find('a')['title']
    Cost = book_container.find('p', class_='price_color').text.strip('£').replace('Â', '')
    Rating = book_container.find('p', class_='star-rating')['class'][1]
    Stock_Availability = book_container.find('p', class_='instock availability').text.strip()
    
    return Title, Cost, Rating, Stock_Availability


def scrape_books_from_page(page_url):
    soup = get_page_content(page_url)
    book_containers = soup.find_all('article', class_='product_pod')
    books = []
    for container in book_containers:
        books.append(scrape_book_details(container))
    return books

    
page_url = url.format(1)


books_on_first_page = scrape_books_from_page(page_url)

for book in books_on_first_page:
    print(f"Title: {book[0]}, Cost: £{book[1]}, Rating: {book[2]} Stock_Availability: {book[3]}")




Title: A Light in the Attic, Cost: £51.77, Rating: Three Stock_Availability: In stock
Title: Tipping the Velvet, Cost: £53.74, Rating: One Stock_Availability: In stock
Title: Soumission, Cost: £50.10, Rating: One Stock_Availability: In stock
Title: Sharp Objects, Cost: £47.82, Rating: Four Stock_Availability: In stock
Title: Sapiens: A Brief History of Humankind, Cost: £54.23, Rating: Five Stock_Availability: In stock
Title: The Requiem Red, Cost: £22.65, Rating: One Stock_Availability: In stock
Title: The Dirty Little Secrets of Getting Your Dream Job, Cost: £33.34, Rating: Four Stock_Availability: In stock
Title: The Coming Woman: A Novel Based on the Life of the Infamous Feminist, Victoria Woodhull, Cost: £17.93, Rating: Three Stock_Availability: In stock
Title: The Boys in the Boat: Nine Americans and Their Epic Quest for Gold at the 1936 Berlin Olympics, Cost: £22.60, Rating: Four Stock_Availability: In stock
Title: The Black Maria, Cost: £52.15, Rating: One Stock_Availability: In

In [4]:
all_books = []

page_number = 1

while True:
    # Construct the URL for the current page
    page_url = url.format(page_number)
    
    # Scrape books from the current page
    full_book_and_containers_on_page = scrape_books_from_page(page_url)

    # If no books are found, break the loop (assuming no books means we've reached the end)
    if not full_book_and_containers_on_page:
        break
    
    # Add the books from the current page to the all_books list
    all_books.extend(full_book_and_containers_on_page)

    print(f"Page {page_number}: Found {len(full_book_and_containers_on_page)} books")
    
    page_number += 1
time.sleep(2)

for book in all_books:
    print(f"Title: {book[0]}, Cost: £{book[1]}, Rating: {book[2]}, Stock: {book[3]}")

Page 1: Found 20 books
Page 2: Found 20 books
Page 3: Found 20 books
Page 4: Found 20 books
Page 5: Found 20 books
Page 6: Found 20 books
Page 7: Found 20 books
Page 8: Found 20 books
Page 9: Found 20 books
Page 10: Found 20 books
Page 11: Found 20 books
Page 12: Found 20 books
Page 13: Found 20 books
Page 14: Found 20 books
Page 15: Found 20 books
Page 16: Found 20 books
Page 17: Found 20 books
Page 18: Found 20 books
Page 19: Found 20 books
Page 20: Found 20 books
Page 21: Found 20 books
Page 22: Found 20 books
Page 23: Found 20 books
Page 24: Found 20 books
Page 25: Found 20 books
Page 26: Found 20 books
Page 27: Found 20 books
Page 28: Found 20 books
Page 29: Found 20 books
Page 30: Found 20 books
Page 31: Found 20 books
Page 32: Found 20 books
Page 33: Found 20 books
Page 34: Found 20 books
Page 35: Found 20 books
Page 36: Found 20 books
Page 37: Found 20 books
Page 38: Found 20 books
Page 39: Found 20 books
Page 40: Found 20 books
Page 41: Found 20 books
Page 42: Found 20 books
P

In [5]:
def savetoDB(Title, Cost, Rating, Stock_Availability):
    try:
        print(f"Attempting to save: {Title}, {Cost}, {Rating}, {Stock_Availability}")
        
        connection = mysql.connector.connect(
            host='localhost',
            port=3306,
            database='bookstoscrape',
            user='root',
            password='Lauraandlunaforever1993!'
        )
        
        if connection.is_connected():
            print("CONNECTED")
            cursor = connection.cursor()
            
            sql = """INSERT INTO bookdata (title, cost, rating, stock_availability) 
                     VALUES (%s, %s, %s, %s)"""
            values = (Title, Cost, Rating, Stock_Availability)
            
            print(f"Executing SQL: {sql}")
            print(f"With values: {values}")
            
            cursor.execute(sql, values)
            connection.commit()
            print(f"Data inserted! Affected rows: {cursor.rowcount}")
            
    except Exception as e:
        print(f"Error details: {type(e).__name__}: {e}")
        import traceback
        traceback.print_exc()
    
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("Connection closed.")
        

In [6]:
for book in all_books:
    print(f"Title: {book[0]}, Cost: {book[1]}, Rating: {book[2]}, Stock: {book[3]}")
    savetoDB(book[0], book[1], book[2], book[3])  # Pass the parameters!

Title: A Light in the Attic, Cost: 51.77, Rating: Three, Stock: In stock
Attempting to save: A Light in the Attic, 51.77, Three, In stock
Error details: ProgrammingError: 1045 (28000): Access denied for user 'root'@'localhost' (using password: YES)


Traceback (most recent call last):
  File "c:\Users\David\AppData\Local\Programs\Python\Python312\Lib\site-packages\mysql\connector\connection_cext.py", line 334, in _open_connection
    self._cmysql.connect(**cnx_kwargs)
_mysql_connector.MySQLInterfaceError: Access denied for user 'root'@'localhost' (using password: YES)

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\David\AppData\Local\Temp\ipykernel_41620\837808129.py", line 5, in savetoDB
    connection = mysql.connector.connect(
                 ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\David\AppData\Local\Programs\Python\Python312\Lib\site-packages\mysql\connector\pooling.py", line 322, in connect
    return CMySQLConnection(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\David\AppData\Local\Programs\Python\Python312\Lib\site-packages\mysql\connector\connection_cext.py", line 151, in __init__
    self.connect(**kwargs)
  Fil

UnboundLocalError: cannot access local variable 'connection' where it is not associated with a value